In [72]:
import urllib.request
import json
import numpy as np
import pandas as pd
import altair
import pprint
import os, subprocess, sys,codecs, locale
import re
import traceback
from collections import namedtuple
import datetime
import pdfplumber
import pickle
import pandas as pd
import openai
from pprint import pprint
from aocutils.datascience import save, load
token = open('.tokens/test.txt','r').read()
openai.organization = ""
openai.api_key = token

# openai.Model.list()

In [73]:
from pathlib import Path
df = pd.read_pickle("moties_for_llm.pickle")
print(len(df))
if Path("motie_summaries.pickle").exists():
    print('Summary file found')
    summaries = pd.read_pickle("motie_summaries.pickle").drop_duplicates('doc_Id',keep='last')
    df = pd.merge(df, summaries, on='doc_Id', how='outer')
    print(f"Amount of moties {len(df)} without summary {df['Summary'].isna().sum()}")
df['len'] = df['Prompt'].str.len()
# df = df.sample(frac=1)

9008
Summary file found
Amount of moties 9008 without summary 0


In [74]:
char2token = 0.4
summarize_factor = 0.21
# calculate amount of tokens
test = df.groupby(['Indienende_partij'])['len'].aggregate([sum, len])
test['tokens'] = round(test['sum'] * char2token * summarize_factor,0)
test['tokenspermotie'] = round(test['tokens'] / test['len'])
test['numsummarysummaries'] = test['len'] // 18
test

,sum,len,tokens,tokenspermotie,numsummarysummaries
Indienende_partij,,,,,
50PLUS,2873,4,241.0,60.0,0
BBB,391537,427,32889.0,77.0,23
BIJ1,142607,125,11979.0,96.0,6
CDA,686888,638,57699.0,90.0,35
ChristenUnie,486545,475,40870.0,86.0,26
D66,765585,756,64309.0,85.0,42
DENK,398857,507,33504.0,66.0,28
FVD,159075,220,13362.0,61.0,12
Fractie Den Haan,129231,138,10855.0,79.0,7


In [78]:
token_count = 0
summary_count = 0
already_done = 0
errors = 0
model = "gpt-3.5-turbo"
def get_sum(row):
    global token_count, summary_count, already_done, errors
    if 'Summary' not in row.index or pd.isna(row.Summary):
        try:
            completion = openai.ChatCompletion.create(
            model = model,
            messages=[
                {"role": "system", "content": "Je bent heel goed in het samenvatten van moties. Je vat moties samen in maximaal 2 zinnen. Dat de Kamer de regering verzoekt laat je weg."},
                {"role": "user", "content": row.ClippedText},
            ]
            )
            token_count += completion['usage']['total_tokens']
            summary_count += 1
            if not summary_count % 10:
                print(summary_count)
            return {'doc_Id': row.doc_Id,
                    'ClippedTextforSummary':row.ClippedText,
                    'SummaryModel': model,
                    'SummaryDateRetrieved' : datetime.date.today(),
                    'Summary': completion.choices[0].message.content, 
                    'SummaryTokens': completion['usage']['completion_tokens']}
        except Exception as e:
            print('call didnt go well',e)
            errors += 1
        
    
    already_done += 1
    return {'doc_Id': row.doc_Id,
        'ClippedTextforSummary':row.ClippedTextforSummary,
        'SummaryModel': row.SummaryModel,
        'SummaryDateRetrieved' : row.SummaryDateRetrieved,
        'Summary': row.Summary, 
        'SummaryTokens': row.SummaryTokens}

output = df.apply(get_sum, axis='columns', result_type='expand')
summaries = pd.merge(summaries, output, how='outer').drop_duplicates('doc_Id', keep='last')
save(summaries, 'motie_summaries.pickle')
print(f'{summary_count} summaries downloaded, with {token_count} total tokens. {already_done} were already summarized earlier. {errors} errors occurred. Total summaries now {len(summaries)}')
df = pickle.load(open("moties_for_llm.pickle","rb"))
df = pd.merge(df, summaries, on='doc_Id', how='outer')
save(df, 'moties_for_llm_incl_moties.pickle')
df.to_csv('motie_samenvattingen.xlsx')



0 summaries downloaded, with 0 total tokens. 9008 were already summarized earlier. 0 errors occurred. Total summaries now 9008


### Using GPT-4 to summarize all motie summaries per party

In [56]:
import urllib.request
import json
import numpy as np
import pandas as pd
import altair
import pprint
import os, subprocess, sys,codecs, locale
import re
import traceback
from collections import namedtuple
import datetime
import pdfplumber
import pickle
import pandas as pd
import openai
from pprint import pprint
from aocutils.datascience import save, load

openai.organization = ""
openai.api_base = "https://openrouter.ai/api/v1"
openroutertoken = open('.tokens/test2.txt','r').read()
openai.api_key = openroutertoken
OPENROUTER_REFERRER = "https://github.com/jvanelteren/kamermoties"

# openai.Model.list()
df = load('moties_for_llm_incl_moties.pickle')

In [57]:
# With the summaries, the amount of tokens is reduced with ~80%
# this is roughly the cost with gpt 4
df['SummaryTokens'].sum() / 1000 * 0.06

29.07138

In [58]:
df = df.sort_values(['Indienende_partij'])
df['CumulativeTokens'] = df.groupby('Indienende_partij')['SummaryTokens'].cumsum()

In [59]:
import math
max_tokens_per_summary = 3300
tokens_party = df.groupby('Indienende_partij')['SummaryTokens'].sum().reset_index()

tokens_party['Amount_tokens_per_summary'] = tokens_party['SummaryTokens'] // ((tokens_party['SummaryTokens'] // max_tokens_per_summary) + 1) + 10
cutoff = 30000
tokens_party['Overshoot'] = (tokens_party['SummaryTokens'] - cutoff).clip(0)
tokens_party['Keep_percentage'] = (cutoff / tokens_party['SummaryTokens']).clip(None, 1)
tokens_party


,Indienende_partij,SummaryTokens,Amount_tokens_per_summary,Overshoot,Keep_percentage
0,50PLUS,151.0,161.0,0.0,1.000000
1,BBB,23908.0,2998.0,0.0,1.000000
2,BIJ1,7060.0,2363.0,0.0,1.000000
3,CDA,40191.0,3101.0,10191.0,0.746436
4,ChristenUnie,29652.0,3304.0,0.0,1.000000
5,D66,45347.0,3249.0,15347.0,0.661565
6,DENK,25736.0,3227.0,0.0,1.000000
7,FVD,9656.0,3228.0,0.0,1.000000
8,Fractie Den Haan,7416.0,2482.0,0.0,1.000000
9,GroenLinks,36989.0,3092.0,6989.0,0.811052


In [54]:
16*1.89

30.24

In [60]:
# remove not relevant parties
peilingen = 'VVD D66 PVV CDA SP PvdA GroenLinks FVD PvdD ChristenUnie Volt JA21 SGP DENK BBB Omtzigt'.split()

for p in df['Indienende_partij'].unique():
    # deze partijen neem ik niet mee in de analyses
    if p not in peilingen:
        print(p)
print(len(df))
df = df[df['Indienende_partij'].isin(peilingen)]        
print(len(df))

50PLUS
BIJ1
Fractie Den Haan
Groep Van Haga
Gündogan
9008
8227


In [61]:

token_count = 0
summary_count = 0
errors = 0
def get_final_summary(text):
    global token_count, errors, summary_count
    try:
        completion = openai.ChatCompletion.create(
        model="openai/gpt-4-32k", #gpt-4-32k
        headers={"HTTP-Referer": OPENROUTER_REFERRER},
        messages=[
              {"role": "system", "content": "Je bent een uitstekende politiek analyst die met eenvoudige woorden kan uitleggen waar een politieke partij voor staat. De input zijn samenvattingen van moties ingediend door de partij, gescheiden door '|'. Je geeft in maximaal 3 begrijpelijke zinnen aan wat de principes van de partij zijn. De laatste zin bevat enkel 5 kernprincipes, gescheiden door komma's"},
              {"role": "user", "content": text},
          ]
        )
        token_count += completion['usage']['total_tokens']
        summary_count += 1
        if not summary_count % 10:
            print(summary_count)
        return completion.choices[0].message.content
    except Exception as e:
        print('call didnt go well',e)
        errors += 1
        return ''


In [62]:
len(df.sample(frac=1, random_state=112))

8227

In [64]:
finals= finals[:3]

In [71]:
# finals = []
tot = 0
# some parties need reduction of number of moties, this is computed as keep_percentage
for partij, group in df.groupby('Indienende_partij'):
        if partij not in ['BBB', 'CDA', 'ChristenUnie']:

                moties_amount = len(group)
                keep = tokens_party.groupby('Indienende_partij').mean()['Keep_percentage'][partij]
                discard = 1 - keep
                tot += (moties_amount)
                # I'm using random sampling to start_amount is not relevant, only if you want to keep the latest moties
                start_amount = round(moties_amount * discard)
                print(f'Now running {partij}, with n={moties_amount}, of which {keep} are kept')
                motie_samenvattingen = ' | '.join(group['Summary'].sample(frac=keep, random_state=10))
                # result = 'hi'
                result = get_final_summary(motie_samenvattingen)
                print(result)

                finals.append({
                        'Partij' : partij,
                        'Summary': result,
                        'Moties': moties_amount,
                        'Keep': keep,
                        'Motie_samenvattingen': motie_samenvattingen
                })
df_final = pd.DataFrame(finals)
save(df_final, 'final_summaries_full_kamer.pickle')
df_final.to_csv('final_summaries_full_kamer.csv')

print(f'Total tokens {token_count}, summaries {summary_count}, errors {errors}')

Now running D66, with n=756, of which 0.6615652634132357% are kept
Deze politieke partij richt zich op een breed scala aan onderwerpen. De partij beziet de maatschappij als een samenhangend geheel en hecht veel waarde aan eerlijke verdeling van welvaart, inclusiviteit, gelijke kansen, en het zorgvuldig omgaan met het milieu. Daarnaast behartigt de partij de belangen van precaire groepen zoals ouderen, studenten, deelnemers aan bijstand, en asielzoekers. Ook is er aandacht voor technologische innovaties en hun invloed op de samenleving.

Kernprincipes: gelijkheid, solidariteit, duurzaamheid, rechtvaardigheid, en inclusiviteit.
Now running DENK, with n=507, of which 1.0% are kept
Deze politieke partij staat voor het beschermen van de belangen van verschillende minderheids- en achtergestelde groepen binnen de samenleving, waaronder mensen met een migratieachtergrond en moslims. De focus ligt op gelijke rechten, diversiteit en inclusie. Daarnaast is er veel aandacht voor het aanpakken van 

In [77]:
df_final.to_excel('final_summaries_full_kamer.xlsx')


In [69]:
finals[2]['Summary']

'Deze politieke partij heeft een sterk sociaal en duurzaam karakter. Ze zetten zich in voor een eerlijke verdeling van welvaart en mogelijkheden, zodat iedereen kan participeren en meedoen in de samenleving. Respect voor andere culturen en religies staat centraal, evenals solidariteit met kwetsbare individuen en gemeenschappen. Ze streven naar een wereld waarin geen plek is voor discriminatie en racisme. Verduurzaming van de economie en energietransitie richting hernieuwbare bronnen zijn ook belangrijke agendapunten. Deze partij gelooft in de kracht van samenwerking en neemt regelmatig initiatief om coalities te vormen rondom sociaal-economische, milieugerelateerde en mensenrechtenkwesties.\n\nKernprincipes: Sociale rechtvaardigheid, Gelijkheid van kansen, Respect voor diversiteit, Solidariteit, Duurzame ontwikkeling.'

### Interpretation

In [9]:
import urllib.request
import json
import numpy as np
import pandas as pd
import altair
import pprint
import os, subprocess, sys,codecs, locale
import re
import traceback
from collections import namedtuple
import datetime
import pdfplumber
import pickle
import pandas as pd
import openai
from pprint import pprint
from aocutils.datascience import save, load

openai.organization = ""
openai.api_base = "https://openrouter.ai/api/v1"
openroutertoken = open('.tokens/test2.txt','r').read()
openai.api_key = openroutertoken
OPENROUTER_REFERRER = "https://github.com/jvanelteren/kamermoties"

# openai.Model.list()
df = load('moties_for_llm_incl_moties.pickle')

df_final = load('final_summaries.pickle')



In [11]:
df_final = df_final.sample(frac=1)

In [21]:
for idx, row in enumerate(df_final.iterrows()):
    if row[1].values[0] == 'CDA':
        print('Partij', str(idx+1), row[1].values[0])
        print(row[1].values[3])

Partij 10 CDA
Het kabinet wordt verzocht de maatschappelijke kosten-batenanalyse niet langer als enige besluitvormingsinstrument in het infrastructuurbeleid te gebruiken en te onderzoeken hoe deze analyse nog waarde kan hebben. Tevens wordt het kabinet verzocht het eenzijdig rendementsdenken te schrappen uit toekomstige analyses en inzichtelijk te maken hoe de analyses worden gebaseerd op bereikbaarheidsnormen. | Maak de groene waterstofroutekaart voor de raffinagesector volwaardig en bied langetermijnzekerheid aan marktpartijen door na 2025 openstelling te bieden voor groene waterstof in Nederland. | Neem bij een bod van de regio voor de Bodegravenboog een werkende parallelstructuur inclusief extra op- en afrit bij Bodegraven mee en stel een routekaart op voor een veilige ongelijkvloerse fietsoversteek bij de op- en afritten van de N11 bij Bodegraven binnen drie jaar. | De Kamer verzoekt de regering periodiek inzicht te geven in de samenstelling van referentiebudgetten van sociale min

In [41]:
df_final = load('final_summaries.pickle')


In [42]:
df_final['Summary'][0]

'De partij zet zich in voor beleid dat duurzaamheid en milieubescherming stimuleert, zoals de promotie van zonnepanelen, de regulering van waterkwaliteit, de bevordering van regionale programmering en de opstelling van een Nationaal Koudeplan. Ze benadrukt de bijdrage van landbouw en visserij aan de Nederlandse economie en pleit voor maatregelen om de nadelige gevolgen van economische en beleidsveranderingen voor deze sectoren te beperken. De partij is ook gericht op sociale en morele vaardigheden in het onderwijs, de waarborging van de digitale euro, en het garanderen van de gezondheidszorg. De partij neemt een sterke houding in tegen staten die betrokken zijn bij terrorisme en pleit voor sancties en diplomatieke maatregelen. Ze staat voor een strikter immigratiebeleid en hecht waarde aan de mening van de bevolking in belangrijke beslissingen, zoals de invoering van de digitale euro. De kernprincipes van de partij zijn het bevorderen van duurzaamheid, het beschermen van de Nederlandse

In [43]:
len('De partij zet zich in voor beleid dat duurzaamheid en milieubescherming stimuleert, zoals de promotie van zonnepanelen, de regulering van waterkwaliteit, de bevordering van regionale programmering en de opstelling van een Nationaal Koudeplan. Ze benadrukt de bijdrage van landbouw en visserij aan de Nederlandse economie en pleit voor maatregelen om de nadelige gevolgen van economische en beleidsveranderingen voor deze sectoren te beperken. De partij is ook gericht op sociale en morele vaardigheden in het onderwijs, de waarborging van de digitale euro, en het garanderen van de gezondheidszorg. De partij neemt een sterke houding in tegen staten die betrokken zijn bij terrorisme en pleit voor sancties en diplomatieke maatregelen. Ze staat voor een strikter immigratiebeleid en hecht waarde aan de mening van de bevolking in belangrijke beslissingen, zoals de invoering van de digitale euro. De kernprincipes van de partij zijn het bevorderen van duurzaamheid, het beschermen van de Nederlandse economie, en het waarborgen van de volksgezondheid en sociale stabiliteit.')

1074

In [7]:
par = [row[1].values[0] for row in df_final.iterrows()]
import numpy
numpy.random.shuffle(par)
par

['JA21',
 'ChristenUnie',
 'CDA',
 'PVV',
 'PvdD',
 'SP',
 'DENK',
 'D66',
 'PvdA',
 'FVD',
 'GroenLinks',
 'VVD',
 'SGP',
 'Omtzigt',
 'BBB',
 'Volt']